In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates


final_rings = {5120, 8448, 9217, 25345, 26113, 20229, 36231, 20105, 22415, 9360, 12432, 31123, 35219, 5397, 33302, 31386, 17309, 19358, 19103, 27167, 33057, 33315, 21288, 27176, 8234, 28328, 34219, 6317, 34347, 35375, 6448, 26365, 20402, 32298, 26164, 33071, 21433, 9404, 22205, 9151, 32064, 6466, 14402, 34115, 14406, 25287, 31176, 3403, 29390, 29266, 9044, 17494, 32342, 25432, 9433, 9435, 22364, 22365, 25051, 28000, 36452, 12262, 26347, 35052, 35435, 16110, 22126, 27374, 22123, 22390, 16379, 20349, 3198}



In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200, 225, 250, 275, 300, 350, 450]
#pixel_values = [450]
survived = []
len_container = []


k = 1
filename = f'/Users/vdk/Software/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
keks = k*1000
for i,j in enumerate(event_iterator):
    if i == 110:
        event = j
        image_processor = ImageProcessor(source.subarray)
        #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
            image_extractor_type="GlobalPeakWindowSum",
            subarray = source.subarray)
        for tel_id in j.r1.tel.keys():
            test_arr = np.where(j.r1.tel[tel_id].waveform > 7)
            #if j.r1.tel[tel_id].waveform.sum() > 1500:
            if len(test_arr[0]) > 50:
                event_container[keks+i] = j
#                     calib(j)
#                     image_processor(j)
#                     muon_processor(j)
#             for m in j.muon.tel.keys():
#                 if not np.isnan(j.muon.tel[m].efficiency.width):
#                     event_container[keks+i] = j
        break


t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
test = np.where(event.r1.tel[tel_id].waveform > 4)[0]

In [ ]:
# Rewrite proton pixel number without hist

# Protons pixel number


pixel_number_arr_p_pure = []
counter = 0

k = 1
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=1000)
event_iterator = iter(source)
count = 0

for k in range(1,2):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=1)
    event_iterator = iter(source)
    for event in event_iterator:
        for tel in event.trigger.tels_with_trigger:
            temp_arr = []
            count += 1
            pixel_mask = np.array(event.simulation.tel[tel].true_image, dtype = bool)
            temp_event = event.r1.tel[tel].waveform[pixel_mask, :]
            print(temp_event.shape)
            #for i in temp_event:
            for i in event.r1.tel[tel].waveform:
                temp_arr.append(np.max(i)) # знаходить максимальне значення в кожному пікселі
            temp_arr = np.array(temp_arr)
            print(len(temp_arr))            
            print(len(temp_arr[temp_arr>0.7]))
            #n, bins, _ = plt.hist(temp_arr, bins = 100, range = (0,15), density = False, alpha = 0.35, label = 'pure muons')
            #hist = np.histogram(temp_arr, bins = 100, range = (0,15))
            #mask = np.array(np.where(hist[1] > r1_cut)[0])
            #pixel_number_arr_p_pure.append(sum(hist[0][mask[:-1]]))


#print("array with number of pixels", len(pixel_number_arr_p_pure))
#print("pure muons = ", len(pure_muons))
#print("counter = ", counter)
#pixel_number_arr_p_pure
#t = np.where(bins[:-1] > 2)[0]
#print("n = ", sum(n[t]))
#print("bins", bins[:-1])
print(count)

In [ ]:
for i,j in enumerate(event.r1.tel[tel_id].waveform):
    print("i = ", i, "j = ", j)

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

#pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200]
pixel_values = [150]
survived = []
len_container = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,11):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(i)) # знаходить максимальне значення в кожному пікселі
                        pixel_number = len(tmp_arr[t_arr>r1_cut])
                        print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
#     for event in event_container:
#         print("thats event = ", event)
#         if event in final_rings:
#             print("we are really counting")
#             counter+=1
            
#     survived.append(counter)
#     len_container.append(len(event_container))
#     print("survived = ", survived)
#     print("len_cont = ", len_container)
#     t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))

In [ ]:
histplot(number_of_active_pixels, bins = 30, stat = 'count')

In [ ]:
len(number_of_active_pixels)

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

other_pixel_values = []


for k in range(1,11):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        if len(j.trigger.tels_with_trigger) > 1:
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
            for tel_id in j.r1.tel.keys():
                test_arr = np.where(j.r1.tel[tel_id].waveform > 4)
                other_pixel_values.append(len(test_arr[0]))
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
histplot(other_pixel_values, bins = 30, stat = 'count')

In [ ]:
min(other_pixel_values)

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

event_num = 8234

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()




for k in range(1,11):
    filename = f'/Users/vdk/Software/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:
            image_processor = ImageProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
            #calib(j)
            #image_processor(j)
            #muon_processor(j)
            #for m in j.muon.tel.keys():
            #    if not np.isnan(j.muon.tel[m].efficiency.width):
            event_container[keks+i] = j
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")


event = event_container[event_num]

In [ ]:
muon_ring_events = [3198, 3403, 6317, 6466, 8234, 8448, 9404, 9433, 12262, 14402, 14406, 16110, 16379, 19103, 20229, 20402, 21288, 21433, 22123, 22205, 22364, 22365, 22390, 22415, 25051, 25345, 26113, 26164, 26347, 26365, 27176, 27374, 28000, 28328, 29266, 29390, 31176, 31386, 32064, 32298, 33057, 33071, 34115, 35052, 35219, 35375, 35435, 36231, 36452]
len(muon_ring_events)

In [ ]:
event_container_fitted = {}
for i in event_container:
    #print(i)
    if i in muon_ring_events:
        image_processor = ImageProcessor(source.subarray)
        #muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
            image_extractor_type="GlobalPeakWindowSum",
            subarray = source.subarray)
        calib(event_container[i])
        image_processor(event_container[i])
        muon_processor(event_container[i])
        for m in event_container[i].muon.tel.keys():
            if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                event_container_fitted[keks+i] = event_container[i]
    t2 = time.perf_counter(), time.process_time()

In [ ]:
event_container_fitted.keys()

In [ ]:
plt.grid(alpha = 0.6)
#plt.yticks([-0.5,0,0.5,1,1.1,1.2,1.3,1.4])
for pix_id in range(0,1855):
    plt.plot(event_container_fitted[13198].r1.tel[1].waveform[pix_id], drawstyle='steps')
print(len(event_container_fitted))

In [ ]:
for i in event_container_fitted:
    for m in event_container_fitted[i].muon.tel.keys():
        if not np.isnan(event_container_fitted[i].muon.tel[m].efficiency.width):
            #print("here")
            plt.ylim(-1,10)
            plt.grid(alpha = 0.6)
            #plt.yticks([-0.5,0,0.5,1,1.1,1.2,1.3,1.4])
            for pix_id in range(0,1855):
                plt.plot(event_container_fitted[i].r1.tel[m].waveform[pix_id], drawstyle='steps')

In [ ]:
wave_sum = []
dl_pix = []
for i in event_container:
    for tels in event_container[i].trigger.tels_with_trigger:
        wave_sum.append(event_container[i].r1.tel[tels].waveform.sum())
        dl_pix.append(event_container[i].dl1.tel[tels].parameters.morphology.n_pixels)
        #print(" wave = ", event_container[i].r1.tel[tels].waveform.sum(), " pix = ", event_container[i].dl1.tel[tels].parameters.morphology.n_pixels)
        
wave_sum = np.array(wave_sum)
dl_pix = np.array(dl_pix)

In [ ]:
plt.scatter([i for i in range(len(dl_pix))], wave_sum)



In [ ]:
plt.scatter([i for i in range(len(dl_pix))], dl_pix)

In [ ]:
event_num = 8234
plt.ylim(-1,2)
plt.grid()
plt.yticks([-0.5,0,0.5,1,1.1,1.2,1.3,1.4])
for pix_id in range(0,1855):
    plt.plot(event_container[event_num].r1.tel[1].waveform[pix_id], drawstyle='steps')

plt.plot(range(40), [1.2 for i in range(40)], color = 'black', lw = 4, label = 'treshold line')
plt.ylabel("Waveform amplitude")
plt.xlabel("Time sample")
plt.title("R1 Waveforms for contaminated muon ring")
plt.legend()
    
#np.mean(event_container[2414].r1.tel[1].waveform)

In [ ]:
#plt.ylim(-1,2)
plt.grid(alpha = 0.6)
#plt.yticks([-0.5,0,0.5,1,1.1,1.2,1.3,1.4])
for pix_id in range(0,1855):
    plt.plot(event_container[event_num].r1.tel[1].waveform[pix_id], drawstyle='steps')

plt.plot(range(40), [2 for i in range(40)], color = 'black', lw = 4, label = 'cut line')

plt.ylabel("Waveform amplitude")
plt.xlabel("Time sample")
plt.title("R1 Waveforms for contaminated muon ring")
plt.savefig('/Users/vdk/R1_cutline.png')
plt.legend()

In [ ]:
test = np.where(event_container[8234].r1.tel[1].waveform > 1.2)
len(test[0])

In [ ]:
83/473

In [ ]:
for pix_id in test[0]:
    plt.plot(event_container[event_num].r1.tel[1].waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
np.mean(event_container[event_num].r1.tel[2].waveform)

In [ ]:
np.argmax(event.r0.tel[1].waveform[1])

In [ ]:
pixes = []
for i,j in enumerate(event.r1.tel[1].waveform):
    if np.argmax(j) < 10:
        pixes.append(i)


In [ ]:
a = 6
for pix_id in pixes:
    plt.plot(event.r1.tel[1].waveform[pix_id], drawstyle='steps')


In [ ]:
sumer = []
mean = []
for pix_id in pixes:
    #print(event.r1.tel[1].waveform[pix_id])
    mean.append(np.mean(event.r1.tel[1].waveform[pix_id]))
    sumer.append(event.r1.tel[1].waveform[pix_id])
    
sumer = np.array(sumer)
mean = np.array(mean)

In [ ]:
np.mean(sumer, axis = 0).shape

In [ ]:
np.mean(np.mean(sumer, axis = 0))

In [ ]:
plt.plot([i for i in pixes], mean)

In [ ]:
len(pixes)

In [ ]:
len(mean)

In [ ]:
len([i for i in pixes])

In [ ]:
np.mean(mean)

In [ ]:
event.trigger.tel[1].time.to_value("mjd", "float")

In [ ]:
test = []
for k,l in enumerate(event.r1.tel[1].waveform):
    if np.argmax(l) == 3:
        test.append(l)

test = np.array(test)

In [ ]:
np.mean(test, axis = 0).shape